<a href="https://www.nvidia.com/dli"> <img src="imgs/header.png" alt="Header" style="width: 400px;"/> </a>

<h1 align="center">인텔리전트 비디오 분석을 위한 딥러닝</h1>
<h4 align="center">(1부)</h4>


<img src="imgs/intro.gif" alt="AFRL1" style="margin-top:50px"/>
<p style="text-align: center;color:gray"> 그림 1. "vehicle" 클래스에 대한 실시간 객체 검출  </p>

*인텔리전트 비디오 분석(Intelligent Video Analytics; IVA)를 위한 딥러닝* 코스에 오신 것을 환영합니다! 

오늘날 수십억 대의 카메라가 매일 엄청난 양의 비디오 데이터를 만들어냅니다. 이 정도 규모에서 다양한 유형의 객체에 대한 식별, 추적, 분할 및 예측과 같은 작업을 위해 특징을 추출하는 것은 간단한 일이 아닙니다. 딥러닝은 빠른 속도와 큰 규모로 데이터를 다루는데 효과적인 방법으로 알려져 있습니다. 소매 마켓의 최적의 판매량을 위한 사람들의 움직임 및 혼잡함을 분석하고, 스마트 주차 관리를 위해 트래픽 분석등이 딥러닝 기반의 인텔리전트 비디어 분석 (IVA) 응용의 예이다. 

본 워크숍에서 여러분은 다음과 같은 내용을 배우게 될 것입니다.

- 하드웨어 가속 디코딩 방법을 사용하여 비디오 피드를 효율적으로 처리하고 준비하는 방법 (랩 1 및 랩 3)
- 딥러닝 모델을 훈련 및 평가하고 "전이 학습 (Transfer learning)" 기법을 활용하여 모델의 효율성과 정확성을 높이며, 데이터 희소성 문제를 완화하는 방법 (랩 2)
- 대규모 비디오 데이터 집합에서 움직이는 객체를 추적하기 위한 고품질 신경망 모델 개발에 수반되는 전략 및 트레이드오프 (랩 2)
- __DeepStream SDK__ 를 사용하여 엔드 투 엔드 가속화 비디오 분석 솔루션 구축 (랩 3)

워크숍을 마치면 주차장 카메라 피드를 기반으로 하드웨어 가속 교통 관리 시스템의 빌딩 블록을 설계, 훈련, 테스트 및 배치할 수 있게 됩니다.


#### 전제 조건
비디오 처리 방법, 딥러닝 모델 및 객체 검출 알고리즘에 대한 지식이 있다면 좋지만 필수적인 것은 아닙니다. 단, 여러분이 프로그래밍의 기초 개념, 특히 __Python__ 과 __C++__ 에 익숙하다고 가정합니다.



#### 쥬피터 노트북에 대하여
시작하기 전에 이 쥬피터 노트북에 대해 알아두어야 할 사항이 있습니다.

1. 노트북은 브라우저에서 렌더링되고 있지만, 내용은 GPU 지원 인스턴스에서 실행되는 대화형 iPython 커널에 의해 스트리밍되고 있습니다.

2. 노트북은 셀로 구성되어 있습니다. 셀은 실행할 수 있는 코드를 포함할 수도 있고, 읽을 수 있는 텍스트 및 이미지를 담고 있을 수도 있습니다.

3. 메뉴에서 ```Run``` 아이콘을 클릭하거나, 키보드 단축키인 ```Shift-Enter```(실행 및 다음 셀 이동) 또는 ```Ctrl-Enter```(실행 및 현재 셀에서 유지)를 이용해 셀을 실행할 수 있습니다.

4. 셀 실행을 중단하려면 툴바의 ```Stop``` 버튼을 클릭하거나 ```Kernel``` 메뉴로 이동하여 ```Interrupt ``` 를 선택하세요.

본 튜토리얼은 다음 주제를 다루고 있습니다.

* [1. 소개](#1)
    * [1.1 정지 영상과 동영상에서의 객체 검출](#1-1)
    * [1.2 Tensorflow 객체 검출 API](#1-2)
    * [1.3 어노테이션 (Annotations)](#1-3)
* [2. 데이터 세트: NVIDIA 인데버(Endeavor) 주차창 데이터 세트](#2)
* [3. 모델을 위한 데이터 준비](#3)
    * [3.1  Raw 어노테이션 데이터를 Pands DataFrame에 넣기](#3-1)
    * [연습 1](#e1)
    * [연습 2](#e2)
* [4. 비디오 데이터 가지고 작업 하기](#4)
    * [4.1 비디오 파일을 프레임 이미지로 바꾸기](#4-1)
    * [연습 3](#e3)
* [5. 추론](#5)
    * [5.1 한 프레임씩 검출하기](#5-1)
    * [5.2 정량적 분석 - Intersection over Union](#5-2)
* [6. 어노테이션을 자르고 정규화하기](#6)
    * [연습 4](#e4)
* [7. TFRecord 파일 생성하기](#7)
    * [7.1 어노테이션과 영상을 TensorFlow Example들로 인코딩하기](#7-1)
    * [7.2 함수들을 연결하여 TFRecord 만들기](#7-2)



<a name="1"></a>
## 1. 소개

<a name="1-1"></a>
### 1.1 정지 영상과 동영상에서의 객체 검출

급격한 교통 카메라의 증가, 자율주행 차량의 전망 확대, "__스마트 시티__"의 유망한 전망에 따라, 보다 빠르고 효율적인 객체 검출 및 추적 모델의 수요가 증가하고 있습니다. 미국인은 하루에 75회 이상 카메라에 잡힐 수 있으며, 그 결과 일주일마다 [40억 시간](https://www.forbes.com/sites/singularity/2012/08/30/dear-republicans-beware-big-brother-is-watching-you/#4317353620da)의 비디오 영상이 처리되고 그 중 상당 부분이 객체 검출 파이프라인을 사용할 가능성이 있습니다!

일반적으로 객체 검출은 정지 영상(프레임)과 동영상 데이터 세트 내에서 사전 정의된 클래스(예: 보행자, 동물, 건물 및 자동차)의 인스턴스를 찾는 과정입니다. 객체 검출 함수는 영상 처리 분야에서 연구가 심도 있게 진행되어 왔음에도 불구하고, 동영상 데이터 및 시간적 정보 관점에서는 덜 다루어졌습니다. 정지 영상에서 객체를 검출하고 분류하기 위해 가장 널리 쓰이는 딥러닝 접근방식은 그 첫번째로 대규모 데이터 집합을 기반으로 딥 네트워크 모델을 학습시키는 것입니다. 이 때 주로 사용되는 데이터는 *ImageNet 또는 Coco* 데이터 셋입니다. 이 단계의 기본 아이디어는 다른 종류, 모델 또는 원래 클래스와 관련된 서브클래스의 시각적 특징을 추출하고 모델링하는 것입니다. 이후의 객체 검출 추론 단계는 관심 영역에 대한 경계 상자 (bounding box) 회귀 (regression) 분석을 수행하고 결과적으로 테스트 정지 영상이나 동영상에 레이블을 붙임으로써 이루어집니다.

프레임별 처리는 딥러닝 기반의 IVA 애플리케이션의 초창기에 많이 사용되었지만, 이후 시간을 고려한 동영상 추적 처리 기법으로 발전되었습니다. 정지 영상과 비교하면, 동영상 데이터 처리는 실시간 데이터 처리 장벽을 해결해야 할 뿐만 아니라 더 많은 컴퓨팅 작업을 필요로 합니다. 또한 동영상에서의 객체는 모션의 blur 현상으로 인해 열화되거나, 가려지거나 (occlusion), 더 낮은 특징 품질을 가질 수 있습니다.

게다가 Raw 데이터를 정보로 전환하는 단계는 데이터의 프로세싱/이용/배포에 있어서 병목으로 작용합니다. 애플리케이션들은 수천시간 분량의 데이터를 처리해야 합니다. 각 프레임을 보고, 연구하여 유용한 정보로 전환해야 합니다. 인공 지능이 이러한 작업을 해야하는 분석석가들의 부담을 줄여 줄 수 있습니다.

본 코스에서는 프레임별 데이터 준비와 객체 검출을 이용하는 IVA의 가장 단순한 접근법으로부터 시작하여, 시간적 객체 추적 모델에 힘입은 동영상 특화 모델까지 살펴볼 것입니다.


<a name="1-2"></a>
### 1.2 TensorFlow 객체 검출 API


이 과정에서는 [TensorFlow 객체 검출 API](https://github.com/tensorflow/models/tree/master/research/object_detection)를 사용합니다. 이 API는 TensorFlow 위에서 구축된 오픈 소스 프레임워크로서 객체 검출 모델을 쉽게 구성하고 학습시켜 배포할 수 있게 해줍니다. 객체 검출 API에는 최근의 딥러닝 발전에 기여한 다섯 가지 검출 모델이 제공됩니다.

1. [MobileNets](https://arxiv.org/abs/1704.04861)을 이용한 Single Shot Multibox Detector ([SSD](https://arxiv.org/abs/1512.02325))
2. [Inception v2](https://arxiv.org/abs/1512.00567)을 이용한 SSD
3. [Resnet](https://arxiv.org/abs/1512.03385) 101을 이용한 [Region-Based Fully Convolutional Networks](https://arxiv.org/abs/1605.06409) (R-FCN) 
4. Resnet 101을 이용한 [Faster RCNN](https://arxiv.org/abs/1506.01497) 
5. [Inception Resnet v2](https://arxiv.org/abs/1602.07261)을 이용한 Faster RCNN 

본 랩에서는 Inception v2을 이용한 SSD, Inception Resnet v2를 이용한 Faster RCNN 및 NasNet에 집중하여 검출기 훈련하고 테스트할 것입니다. 우리는 과적합 (Overfitting)과 데이터 편차 문와 같은 몇 가지 함정에 대해 잘 알고 있어야만 합니다. 다음 랩에서는 `Pandas` 파이썬 패키지를 사용하여 대량의 데이터를 처리하는 방법에 대해 자세히 알아보겠습니다.

IVA 데이터 스트림에서 객체를 검출하는 작업과 관련하여, 각 모델은 각각의 장단점을 가지고 있기 때문에 배포 가능한 시스템을 개발할 때에 이를 잘 고려해야 합니다. 예를 들어, GPU가 있을 경우에 SSD는 통상적인 비디오 프레임 속도(25~30fps)로 데이터를 처리할 수 있습니다. 정확도가 만족할 만한 수준이기는 하지만 학습에 사용하는 데이터의 양과 다양성에 따라 __많은 False Negatives와 False alarms__ 를 발생시킬 수 있습니다. 이와는 대조적으로, NasNet은 훈련 데이터를 적게 사용하여 매우 정확한 검출을 하는 대신에 처리 속도를 그만큼 소요합니다. 보통, GPU를 이용해서 한 자릿수(또는 그 이하)의 fps 성능을 가집니다.


아래에서 이 세 가지 유형의 모델을 간략히 살펴보기로 합니다.


#### Single-Shot Multibox Detector (SSD)

R-CNNs (Region-based Convolutional Neural Networks)의 도입으로 인해 검출 연산은 두 가지 서브작업으로 나뉘어졌습니다.

- __지역화 (Localization)__: 잠재적 객체의 좌표를 반환하기 위해 회귀 분석을 적용하는 프레임(이미지) 내의 위치를 의미합니다. 네트워크는 정답 경계 상자를 이용하여 훈련되고, L2 거리를 사용하여 정답 좌표와 회귀 좌표 사이의 손실값을 계산합니다.

- __분류 (Classification)__: 주어진 프레임에 모델이 훈련받은 클래스 중 하나로 레이블을 붙이는 작업입니다.

SSD (Single-Shot Multibox Detector Network)는 경계 박스 지역화와 분류를 결합하여 한 번의 순방향 네트워크 흐름만으로 수행합니다. SSD는 VGG-16 아키텍쳐를 기반으로 구현되어 있는데, 기존의 완전 연결 레이어 (Fully-Connected Layer)는 새로운 콘볼루션 (Convolution) 특징 추출 레이어으로 대체되었고, 각 레이어는 일련의 *k* 개의 경계 상자(*사전* 정보에 기반함)와 경계 상자 좌표를 출력합니다. 아래에서 SSD 네트워크 아키텍쳐를 확인하십시오.


<img src="imgs/ssd.jpg" alt="SSD" style="width: 800px;"/>
<p style="text-align: center;color:gray"> 그림 2. SSD 아키텍쳐</p>


#### Faster-RCNN

Faster-RCNN은 SSD에 비해 더 많은 컴포넌트와 세부 요소를 가지고 있어서 아키텍쳐가 더 복잡합니다.
SSD 망과는 달리, Faster-RCNN에서는 지역화 및 분류 작업이 서로 다른 망에서 수행됩니다. 지역화 망은 RPN (Region Proposal Network) 라고 하는데, 그 출력은 클래스 유형이 "전경 (Foreground)"과 "배경 (Background)"인 소프트맥스 레이어로 구성됩니다. 두 번째 출력은 제안된 "Anchors"의 회귀자 (Regressor)입니다. 다음으로, RPN 네트워크의 출력과 함께 원래의 특징 맵을 두 번째 망에 입력하고 여기에서 실제 클래스 레이블이 생성됩니다.

<img src="imgs/RCNN.jpg" alt="RCNN" style="width: 800px;"/>
<p style="text-align: center;color:gray"> 그림 3. Faster R-CNN 아키텍쳐</p>


#### NasNet

[NasNet](https://ai.googleblog.com/2017/11/automl-for-large-scale-image.html)은 지금까지 구축된 가장 정확한 모델 중의 하나로, ImageNet 검증 단계에서 82.7%를 기록하여 이전의 모든 인셉션(Inception) 모델보다 높은 값을 기록했습니다. NasNet은 AutoML이라는 접근 방식을 사용하여 주어진 데이터 세트와 잘 작동하는 레이어를 찾습니다. NasnNet의 경우에는 COCO와 ImageNet에 AutoML을 적용한 결과가 결합되어 NasNet 아키텍처를 형성합니다.

본 코스의 후반부에서는, 정확성과 성능을 모두 높이기 위해 특징의 시간적 상관관계를 이용하여 보다 진보된 시스템을 개발하는 방법을 볼 것입니다. 또한 여러 비디오 스트림에 대한 시스템 확장을 가능하게 하기 위해 [__DeepStream__](https://developer.nvidia.com/deepstream-sdk)을 이용할 것입니다.


<img src="imgs/nas.jpg" alt="RCNN" style="width: 600px;"/>
<p style="text-align: center;color:gray"> 그림 4. AutoML 을 이용한 강화 학습 네트워크 선택</p>


<a name="1-3"></a>
### 1.3 어노테이션 (Annotations)

여러분은 종종 훈련과 테스트 샘플의 수를 증가시켜 모델을 훈련하고 평가해야 할 것입니다. 그러기 위해서는 ground-truth 데이터를 확장해야 합니다. 여기에 사용할 수 있는 몇 가지의 비공개 및 오픈 소스 이미지 마크업 도구들이 있습니다. 본 코스에 사용된 모든 동영상은 `Vatic`을 사용하여 어노테이션을 합니다. 이 도구에 대한 자세한 내용은 [웹사이트](http://www.cs.columbia.edu/~vondrick/vatic/)를 참조하십시오.


<br/>

<img src="imgs/vatic.jpg" alt="Vatic imaging" style="width: 800px;"/>
<p style="text-align: center;color:gray"> 그림 5. Vatic 어노테이션 도구 </p>

어노테이션에 대해서는 온톨로지 (Ontology)와 분류 체계 (Taxonomy)를 주의 깊게 고려해야 합니다. 나중에 다른 객체 유형을 쉽게 추가할 수 있도록 유연해야 합니다. 본 랩에서는 분할과 픽셀 수준의 분류는 다루지 않고 오직 객체 검출만을 다룰 것입니다. 객체 분할을 위한 딥러닝 랩을 포함하여 다른 DLI 랩에서 설명한 기법을 사용하여 달성할 수 있습니다. 그러나 이러한 작업을 수행해서 모델을 훈련시키기 위해 여러분의 데이터에도 유사한 방식으로(경계 상자, 다각형, 마스크) 레이블을 붙여야 합니다. 따라서 어떤 레이블링 도구를 사용할지를 결정할 때에는 작업의 최종 목표를 고려해야 합니다.

<br /><br />

이제 다음 섹션에서 데이터 집합을 소개하는 것으로 추론 작업을 시작하겠습니다.



<a name="2"></a>
## 2. 데이터 세트: NVIDIA 인데버(Endeavor) 주차장 데이터 세트

NVIDIA에서는 NDIVIA 본사를 인데버라고 부르는데, 본 코스에서는 이곳 주차장에서 녹화한 비디오 파일을 사용합니다. 비디오 파일은 전방향 카메라를 사용하여 녹화되며, 결과적으로 Raw 비디오 파일에서 모든 직선은 곡선으로 표현되어 있어서 우리의 비디오 처리 작업에 적합하지 않습니다. 이러한 비디오로 작업을 하려면 언워핑(unwarping)을 해야 하는데 우리가 사용할 비디오는 이미 전처리가 되어 있어서 바로 사용하기만 하면 됩니다. 본 코스의 후반부에서 DeepStream SDK를 가지고 작업할 때, 우리가 만들 파이프라인의 일부인 비디오를 언워핑하는 방법을 배울 것입니다. 

<img src="imgs/360.png" alt="Vatic imaging"/>
<p style="text-align: center;color:gray"> 그림 6. 360도 카메라 녹화 샘플과 DeepStream Gst-nvdewarper 플러그인을 사용한 언워핑 결과</p>

인데버 주차장 데이터 집합의 어노테이션은 JSON 형식으로 되어 있습니다. 각 항목은 비디오 안에 등장하는 각 자동차에 대한 고유 인덱스 값인  `track_id`로 시작합니다. track_id를 통해 경계 상자의 집합과 각 경계 상자의 위치를 알 수 있습니다. 아래에서 어노테이션이 어떤 값들을 가지는지 보겠습니다.

__track_id__ : 각 차량의 고유 ID
> __boxes__ : 경계 상자의 집합과 한 프레임 내에서의 각 경계 상자의 위치 정보
> > __frame_id__ : 프레임 번호를 나타내는 일련의 정수값
> > > __attributes__ : 차량 제조업체, 모델, 색상, 주차 상태를 나타내는 *임의의* 속성 집합<br />
> > > __occluded__ : 차량이 완전히 보이는지 다른 객체에 가렸는지를 나타냄<br />
> > > __outside__ : 차량이 프레임 경계 안쪽에 있는지 바깥에 있는지를 나타냄<br />
> > > __xbr__ : 프레임 크기에 상대적으로 나타낸 경계상자의 우하단 x 좌표값. [0, frame width] 범위.<br />
> > > __xtl__ : 프레임 크기에 상대적으로 나타낸 경계상자의 좌상단 x 좌표값. [0, frame width] 범위.<br />
> > > __ybr__ : 프레임 크기에 상대적으로 나타낸 경계상자의 우하단 y 좌표값. [0, frame height] 범위.<br />
> > > __ytl__ : 프레임 크기에 상대적으로 나타낸 경계상자의 좌상단 y 좌표값. [0, frame height] 범위.<br />



아래에서 샘플 비디오에 대한 JSON 파일의 스냅샷을 볼 수 있습니다.

<img src="imgs/json_structure.png" alt="Vatic imaging"/>
<p style="text-align: center;color:gray"> 그림 7. JSON 어노테이션 파일의 스냅샷  </p>

이제 이 코스에서 사용할 라이브러리를 가져옵시다.


In [0]:
#%matplotlib notebook
%matplotlib inline
import pylab as pl
pl.rcParams['figure.figsize'] = (8, 4)
import os, sys, shutil
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import io
import base64
from IPython.display import HTML
from IPython.display import clear_output
from IPython import display
import matplotlib.patches as patches
from matplotlib.pyplot import cm 
import time
import cv2
import pickle
import json
import sort

from os.path import join

from mpl_toolkits.mplot3d import Axes3D

        
import pandas as pd

본 코스에서는 설정 파일을 활용하여 __data__ 속성에 접근합니다. 또한, __models__ 속성을 참조하는 다른 설정 파일들도 사용할 것입니다.


In [0]:
import configparser
config = configparser.ConfigParser()
config.sections()
config.read("utils/iva.conf")
config = config["General"]

모델을 만드는 데 필요한 데이터의 유형을 파악하기 위해 Raw 데이터 중 일부를 살펴봅시다. 쉽게 볼 수 있도록 Raw 비디오를 작게 만들었습니다. 우리 환경에서 재생이 잘 되도록 비디오의 크기를 조절하고 프레임 속도를 줄였음을 유념하십시오.


In [0]:
def disp_video(fname):
    import io
    import base64
    from IPython.display import HTML
    video = io.open(fname, 'r+b').read()
    encoded = base64.b64encode(video)
    return HTML(data='''<video alt="test" width="640" height="480" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

In [0]:
mp4_path = 'imgs/sample.mp4'
print ("Loading video...")
disp_video(mp4_path)

이들 중 하나를 검사하여 Raw 데이터가 어떻게 생겼는지 살펴봅시다.


In [0]:
%%bash
head -c 1000 /dli/data/videos/126_206-A0-3.json

<a name="3"></a>
## 3. 모델을 위한 데이터 준비

TensorFlow 객체 검출 API를 활용하고 관련 KPI를 측정하기 위해서는 Raw 데이터를 `Pandas DataFrame` 객체로 변환해야 합니다. 그 후에, 추론을 위해 이미지와 어노테이션을 결합하고 모델의 정확도를 측정할 수 있습니다.


<a name="3-1"></a>
### 3.1 Raw 어노테이션 데이터를 Pandas DataFrame에 넣기

본 코스의 후반부에서, 모델 훈련을 위해 TensorFlow 레코드 파일, 즉 TFRecord로 데이터를 변환해야 합니다. 이 파일들은 기록 중심의 이진 파일이며 TensorFlow 프로세스가 쉽게 사용할 수 있습니다. TFRecord 특징은 이미지 프레임 및 해당 프레임과 관련된 모든 어노테이션을 단일 행으로 인코딩합니다. 그러나 주차장 데이터 세트에 제공된 어노테이션 데이터는 frame_id가 아닌 track_id에 의해 구성됩니다. 이 차이점으로 인해 데이터를 분류하고 정리한 후 TFRecord로 변환해야만 합니다. Pandas는 이러한 데이터 전처리 단계를 구현하는 일에 매우 유용합니다.

다음에 나오는 연습에서는 한 비디오에서 얻은 데이터만을 가지고 작업할 것입니다. 여기에는 프레임과 메타데이터가 포함됩니다. 이 정도 양의 데이터만으로도 훈련이 가능은 하지만 나중에는 더 많은 데이터를 이용하여 미리 학습된 모델을 사용해 작업해 볼 것입니다. 이 파일과 모델은 시간을 절약하기 위해 미리 만들어 두었습니다.

이제 Raw 어노테이션 파일을 DataFrame에 넣는 방법을 살펴봅시다.


In [0]:
with open(config["JSON_Sample"], 'r') as f: 
    data = pd.read_json(f)

텍스트 파일 안에 들어 있던 모든 데이터는 이제 데이터 변수 안에 들어 있습니다. 이 데이터를 어떻게 다루는지 알아봅시다. 우리는 단순히 데이터를 출력하거나, 출력의 양을 줄이기 위해 head() 함수를 사용하여 DataFrame의 처음 몇 행만을 표시할 수도 있습니다.


In [0]:
print(data.iloc[0].head())

Vatic 어노테이션 도구를 사용함에 따라 어노테이션 파일에는 중복 데이터가 있습니다. 예를 들어, 차량이 프레임을 벗어났음에도 경계 상자 어노테이션은 비디오가 모두 끝날 때까지 유지됩니다! 이런 유형의 어노테이션을 걸러내는 유일한 방법은 차가 시야를 벗어나자마자 `1`로 설정되는 `outside` 속성을 활용하는 것입니다. 이러한 어노테이션들은 훈련이나 평가에 아무런 도움이 되지 않으므로 안심하고 제거해도 됩니다.

주목해야 할 또 다른 문제점은 프레임의 중복입니다. 많은 프레임들에 자동차가 전혀 없습니다! (카메라에 따라 다르지만) 아무런 움직임 없이 주차된 차량을 포함한 프레임은 더욱더 많습니다. 이러한 프레임을 훈련 데이터 세트에 포함시키면 중복/편향 샘플이 생성되고 훈련 품질에 부정적인 영향을 미칠 수 있습니다. 이 문제를 극복하기 위해 우리는 움직이는 차량이 있는 프레임만 사용하고 나머지는 무시하겠습니다.

<img src="imgs/similars.jpg" alt="Vatic imaging"/>
<p style="text-align: center;color:gray"> 그림 8. 어노테이션이 있는 프레임의 중복</p>

다음의 코드 스니펫은 위에서 언급한 문제를 처리하고 움직이는 차량만 포함하는 프레임 목록을 만듭니다.


In [0]:
tracks = data.keys()
frames_list = []
frame_existance = np.zeros(15000)

for i in range(len(tracks)):
    boxes = data[list(tracks)[i]]["boxes"]
    {frames_list.append(k) for k, v in boxes.items() 
                      if v['outside'] == 0 and 'Moving' in v['attributes'] and k not in frames_list}
    
for i in frames_list:
    frame_existance[int(i)] = 1

최종 프레임 세트를 살펴봅시다.

- 움직이는 차량을 포함하는 프레임
- `outside` 속성이 `1`인 프레임은 제거됨

In [0]:

y_pos = np.arange(len(frame_existance))
pl.rcParams['figure.figsize'] = (18, 3)
 
plt.bar(y_pos, frame_existance, align='center', alpha=0.5)
plt.yticks([])

plt.title('Frame indices that include moving cars')
 
plt.show()

프레임을 보면 전체 프레임 중 아주 일부만 움직이는 차량을 포함하고 있는 것을 알 수 있습니다. 이제 처리할 프레임의 수는 급격하게 줄어들었습니다.


<a name="e1"></a>
### 연습 1:

아래에서 움직이는 차량을 포함한 프레임의 비율을 계산하십시오.


In [0]:
# 여기에서 코딩하세요.

해답을 보려면 [여기를](#a1) 보세요.

앞서 살펴본 것처럼 최상위 수준의 어노테이션은 `track_id` 필드입니다. 하위 수준에서 각 경계 박스는 프레임 번호로 분류됩니다. 이 구조를 평평하게 만들어야만 __DataFrame__ 을 더 잘 이해할 수 있고 조작하기도 쉬워집니다. 또한 제공된 경계 상자는 다른 프레임 크기의 어노테이션을 가지고 있기 때문에 수정할 필요가 있습니다. 어노테이션으로 작업된 프레임은 `(611, 480)` 크기를 가집니다. 제공된 비디오의 프레임 크기를 살펴보십시오.


In [0]:
# get video frame size
input_video = cv2.VideoCapture(config["Video_Sample"])
retVal, im = input_video.read()
size = im.shape[1], im.shape[0]
input_video.release()
print("Video frame size (width, height):", size)

다음 코드는 DataFrame을 평탄하게 하고 기존 프레임 크기에 따라 경계 상자를 정규화합니다. 이것은 시간이 많이 소요되는 과정이므로 처리할 트랙의 수를 `1`로 제한했습니다. (대신, 처리된 데이터를 텍스트 파일로부터 읽을 것 입니다.). 동작하는 예제를 보려면 코드를 모두 선택하고 `Ctrl + /`를 눌러 커멘트를 없앤 후, 코드를 실행하십시오.


In [0]:
# print("processing length:", len(frames_list))
# annotated_frames = pd.DataFrame()
# ANNOTATE_SIZE = (611, 480)
# limit = 1 #set this limit to avoid timely DataFrame generation

# if len(frames_list) > 0:
#     for i in range(len(tracks)):
        
#         # remove the following line if the DataFrame is not read from CSV file
#         if i == limit: break
#         boxes = data[list(tracks)[i]]["boxes"]
#         print("\rprocessing track no: {}".format(i), end = '')
#         for k, v in boxes.items():
            
#             if k in frames_list:#  and v['outside']!=1:
#                 # resizing the annotations
                
#                 xmin, ymin, xmax, ymax = v["xtl"], v["ytl"], v["xbr"], v["ybr"]
#                 xmin = int((float(xmin) / ANNOTATE_SIZE[0]) * size[0])
#                 xmax = int((float(xmax) / ANNOTATE_SIZE[0]) * size[0])
#                 ymin = int((float(ymin) / ANNOTATE_SIZE[1]) * size[1])
#                 ymax = int((float(ymax) / ANNOTATE_SIZE[1]) * size[1])
                
                
#                 annotated_frames = annotated_frames.append(pd.DataFrame({
#                     "frame_no": int(k),
#                     "track_id": [list(tracks)[i]],
#                     "occluded": [v["occluded"]],
#                     "outside": [v["outside"]],
#                     "xmin": [xmin],
#                     "ymin": [ymin],
#                     "xmax": [xmax],
#                     "ymax": [ymax],
#                     "label": ['vehicle'],
#                     "attributes": [','.join(v["attributes"])],
#                     "crop": [(0,0,0,0)],
#                     "camera": config["Test_Video_ID"]
#                 }), ignore_index=True)

우리는 프레임을 오프라인에서 처리해서 텍스트 파일에 기록했습니다. 다음 단계는 DataFrame을 읽어들이는 것입니다.


In [0]:
import ast
annotated_frames = pd.read_csv(config['Path_To_DF_File'], converters={2:ast.literal_eval})

In [0]:
print("Length of the full DF object:", len(annotated_frames))
annotated_frames.head()

어노테이션을 단 프레임에는 제거해야 할 *outside* 차량이 포함됩니다.


In [0]:
occluded_filter = annotated_frames["outside"] == 0
annotated_frames = annotated_frames[occluded_filter]

In [0]:
annotated_frames.head()

데이터 세트에서 "occluded"로 표시된 객체의 수를 찾아봅시다. 부울 필터를 사용하면 편리합니다.


In [0]:
occluded_filter = annotated_frames["occluded"] == 1
occluded_only = annotated_frames[occluded_filter]
print ('Total number of occluded objects: {}'.format(len(occluded_only)))
occluded_only.head()

<a name="e2"></a>
### 연습 2:

`annotation_frames` 객체에는 우리가 사용한 데이터 컬럼 외에도 구조화되지 않은 레이블이 몇 개 포함되어 있습니다. `attributes` 컬럼에 이런 값이 들어 있습니다. 이들 값 중 하나가 차량 유형(세단, SUV 등)입니다. 이러한 차량 중 세단 차량이 몇 대인지 확인해 보십시오.


In [0]:
#여기에서 코딩하세요.

해답은 [여기에](#a2) 있습니다.

<a name="4"></a>
## 4. 비디오 데이터 가지고 작업하기

앞서 보았듯이 일부 차량은 화면에 비해 크기가 작습니다. 또한 비디오는 정사각형이 아닌 비율을 가집니다. 이런 것들은 나중에 우리가 훈련 준비를 할 때 명심하고 고려해야 할 사항들입니다.


<a name="4-1"></a>
### 4.1 비디오 파일을 프레임 이미지로 바꾸기

객체 검출 모델은 프레임 기반 데이터에서 작동하기 때문에, 우리는 원래의 동영상 파일에서 프레임을 생성해야 합니다. 그러기 위해, OpenCV를 사용하여 비디오 파일을 엽니다. mp4 동영상 파일을 사용할 것입니다. 그리고 여기서는 어노테이션이 있는 모든 프레임을 저장할 것이지만 매 n 번째 프레임만 골라서 저장할 수 있는 방법을 여러분이 스스로 찾아낼 수 있는지도 확인해 볼 것입니다.


비디오 프레임을 `jpg` 이미지로 변환하면서 어노테이션을 경계 박스로 표시하는 동영상도 만들 것입니다.



In [0]:

colors = [(255, 255, 0), (255, 0, 255), (0, 255, 255), (0, 0, 255), (255, 0, 0), (0, 255, 0), (0, 0, 0), (255, 100, 0), (100, 255, 0), (100, 0, 255), (255, 0, 100)]

def save_images(video_path, image_folder, frames_list, annotated_frames,  video_out_path = '', fps=10):

    if not os.path.exists(image_folder):
        print("Creating image folder")
        os.makedirs(image_folder)
        
    input_video = cv2.VideoCapture(video_path)
    retVal, im = input_video.read()
    size = im.shape[1], im.shape[0]
    fourcc = cv2.VideoWriter_fourcc('h','2','6','4') 
    output_video = cv2.VideoWriter(video_out_path, fourcc, fps, size)

    if not input_video.isOpened():
        print("Sorry, couldn't open video")
        return

    frameCount = 0
    index_ = 1
    
    while retVal:
        
        #print("\r Processing frame no:", frameCount, end = '')
        if str(frameCount) in frames_list:
            print("\rsaving frame no:{}, index:{} out of {}".format(frameCount,index_,len(frames_list)), end = '')
            
            cv2.imwrite(join(image_folder, '{}.jpg'.format(frameCount)), im)
            
            index_ += 1
            #print("frame:",'{}.jpg'.format(frameCount))
            frame_items = annotated_frames[annotated_frames["frame_no"]==int(frameCount)]
            for index, box in frame_items.iterrows():
                #print(box["crop"])
                xmin, ymin, xmax, ymax = box["xmin"], box["ymin"], box["xmax"], box["ymax"]
                xmin2, ymin2, xmax2, ymax2 = box["crop"][0], box["crop"][1], box["crop"][2], box["crop"][3]
                cv2.rectangle(im, (xmin, ymin), (xmax, ymax), colors[0], 1)
                cv2.rectangle(im, (int(xmin2), int(ymin2)), (int(xmax2), int(ymax2)), colors[1], 1)
            output_video.write(im)

        retVal, im = input_video.read()
        frameCount += 1

    input_video.release()
    output_video.release()
    return size        

우리 비디오 샘플에 대하여 아래 함수를 호출하면 실행이 완료되기까지 시간이 좀 걸릴 것입니다.


In [0]:
save_images(config["Video_Sample"], 
            '{}/images/{}'.format(config["Base_Dest_Folder"], config["Test_Video_ID"]),
            frames_list,
            annotated_frames,
            '{}/videos/{}.mp4'.format(config["Base_Dest_Folder"], config["Test_Video_ID"]))

frame_no를 기준으로 프레임을 정렬하고 전체 장면에서 고유 차량의 수를 추출해 봅시다.


In [0]:
annotated_frames = annotated_frames.sort_values(by=['frame_no'])
print("Number of unique track IDs in the video:", annotated_frames['track_id'].nunique())


또한 각 대상 클래스(이 경우 차량)에 있는 평균 픽셀을 확인할 수도 있습니다. 이것은 각 어노테이션과 연관된 경계 상자 좌표를 사용한 간단한 면적 계산입니다. 도표를 통해 각 "track_id"에 대한 평균 면적의 히스토그램 분포를 보여줍니다.


In [0]:
import matplotlib.pyplot as plt

def calc_targ_area(row):
    area = (row['xmax'] - row['xmin']) * (row['ymax'] - row['ymin'])
    row['area'] = area
    return row

#filter for frames that include items
inside_items = annotated_frames[annotated_frames['outside']==0]

# Group the data by label and calculate the area for each annotation of that type
label_groups = inside_items.groupby(['track_id']).apply(calc_targ_area)
label_groups = label_groups.groupby(['track_id']).mean()


# Build up and view a histogram
y_pos = np.arange(len(label_groups))
plt.bar(y_pos, label_groups["area"], align='center', alpha=0.5)
plt.title('Average area of each vehicle in the video')
plt.xlabel("Track ID")
plt.ylabel("Area")
plt.show()


<a name="e3"></a>
### 연습 3

데이터를 좀 더 조사하십시오. 경계 상자의 평균 너비와 높이를 찾아보면 흥미로울 것입니다.


In [0]:
# 여기에서 코딩하세요.

annotated_frames.head()

해답은 [여기에](#a3) 있습니다.    

<a name="5"></a>
## 5. 추론


<a name="5-1"></a>
### 5.1 한 프레임씩 검출하기

훈련 알고리즘을 실행하기 전에 ResNet, NasNet 및 SSD와 함께 Faster RCNN에 대한 추론 프로세스를 살펴볼 것입니다. 여기서 우리는 AVI 데이터 프레임 안에 있는 객체를 검출하기 위해 최적화된 모델인 추론 그래프를 사용할 수 있습니다. 아래 함수들에서 우리는 그래프를 로드하고, 세션을 생성한 후, 망을 순방향으로 따라 내려가는 루프를 돕니다. TensorFlow 그래프는 모델의 작동 간 종속성을 정의하며 TensorFlow 세션은 하나 이상의 장치에 걸쳐 그래프의 일부를 실행합니다. 그래프 및 세션에 대한 자세한 내용은 TensorFlow 문서를 참고하십시오. 또한 알고리즘은 사전 정의된 제안 수에 대한 점수, 경계 상자 위치 및 클래스를 제공합니다. 제안하는 개수는 모델의 설정 파일에서 변경할 수 있는데, 수를 줄이면 성능은 향상되지만 모델의 정확성에 부정적인 영향을 미칠 수 있습니다.

우리는 또한 주어진 프레임에 대한 ground-truth 데이터를 추출하는 함수를 만듭니다. 이 함수를 사용하여 ground-truth 데이터와 추론된 데이터를 비교할 수 있습니다.


In [0]:
def get_info_from_DF(frame_no):
    result = []
    temp = annotated_frames[annotated_frames["frame_no"] == frame_no]
    for i, box in temp.iterrows():
        result.append([int(box["xmin"]), int(box["ymin"]), int(box["xmax"]), int(box["ymax"])])
    return result

In [0]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
def detect_frames(path_to_graph, path_to_labels,
                  data_folder, video_path, min_index, max_index, frame_rate, threshold):
    # We load the label maps and access category names and their associated indicies
    label_map = label_map_util.load_labelmap(path_to_labels)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)

    # Import a graph by reading it as a string, parsing this string then importing it using the tf.import_graph_def command
    print('Importing graph...')
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(path_to_graph, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

    # Generate a video object
    fourcc = cv2.VideoWriter_fourcc('h','2','6','4') 

    print('Starting session...')
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            # Define input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            frames_path = data_folder
            
            
            num_frames = max_index - min_index
    
            reference_image = os.listdir(data_folder)[0]
            image = cv2.imread(join(data_folder, reference_image))
            height, width, channels = image.shape 
            out = cv2.VideoWriter(video_path, fourcc, frame_rate, (width, height))
            print('Running Inference:')
            total_time = 0
            for fdx, file_name in \
                    enumerate(sorted(os.listdir(data_folder),  key=lambda fname: int(fname.split('.')[0]) )):
                
                if fdx<=min_index or fdx>=max_index:
                    continue;
                image = cv2.imread(join(frames_path, file_name))
                image_np = np.array(image)
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
                bboxes = get_info_from_DF(int(file_name.split('.')[0]))
                # Actual detection.
                tic = time.time()
                (boxes, scores, classes, num) = sess.run(
                    [detection_boxes, detection_scores, detection_classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})
                toc = time.time()
                t_diff = toc - tic
                total_time = total_time + t_diff
                # Visualization of the results of a detection.
                vis_util.visualize_boxes_and_labels_on_image_array(
                    image,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=2,
                    min_score_thresh= threshold)

                
                cv2.putText(image, 'frame: {}'.format(file_name), (30, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255))
                            
                for bbox in bboxes:
                   cv2.rectangle(image, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 0, 255), 2)
                   cv2.putText(image, 'FPS (GPU Inference) %.2f' % round(1 / t_diff, 2), (30, 60),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255))
                    
                    
                prog = 'Completed %.2f%% in %.2f seconds' % ((100 * float(fdx - min_index + 1) / num_frames), total_time)
                print('\r{}'.format(prog), end = "")
                cv2.imwrite("data/temp/{}.jpg".format(fdx), image)
                out.write(image)
        out.release()

앞에서 검토한 세 가지 모델, "RCNN", "SSD", "NasNet"의 변형을 분석해 보겠습니다. [TensorFlow Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models)에서 여러가지 훈련된 모델을 검토해 볼 수 있는데, COCO 데이터 집합으로 훈련 중에 나온 모델의 체크포인트도 다운로드할 수 있습니다.

`COCO-trained models`라는 제목의 섹션을 방문하여 `speed` 와 `Mean Average Precision` 측정값을 비교해 보십시오. 우리는 다음에 세 가지 모델을 시험할 것입니다. 세 가지 모델 중에서 어떤 모델이 가장 빠른지 알아보세요. 또한 어떤 모델이 가장 정확합니까?


In [0]:
models = {'faster_rcnn_resnet_101': '/dli/data/tmp/faster_rcnn_resnet/frozen_inference_graph.pb' ,
          'nasnet': '/dli/data/tmp/faster_rcnn_nas/faster_rcnn_nas_coco_2018_01_28/frozen_inference_graph.pb',
          'ssd_mobilenet_v2':'/dli/data/tmp/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb'}

image_folder = '{}/images/{}'.format(config["Base_Dest_Folder"], config["Test_Video_ID"])
model_name = 'faster_rcnn_resnet_101'
PATH_TO_LABELS = config["Path_To_COCO_Labels"]
PATH_TO_DATA = image_folder
VIDEO_OUT_PATH = 'imgs/inference_COCO.mp4'

다음으로, 우리는 각 모델에 대한 추론 과정을 실행하고, 당분간은 그 결과를 시각적으로 비교하려고 노력할 것입니다. `detect_frames` 함수를 호출할 때 추론을 위한 최소 및 최대 프레임 인덱스 값을 넘겨줄 수 있는데, 여기에서는 시간 절약을 위하여 "100"에서 "200"으로 설정했습니다.
또한 네트워크 출력을 신뢰도에 따라 컷아웃 할 수 있는 임계값을 줄 수도 있는데, 기본적으로 0.5로 설정되어 있습니다.


In [0]:
# select model name: possible values:
#                                        'faster_rcnn_resnet_101'
#                                        'nasnet'
#                                        'ssd_mobilenet_v2'

model_name = 'faster_rcnn_resnet_101'
detect_frames(models[model_name], PATH_TO_LABELS, PATH_TO_DATA, VIDEO_OUT_PATH, 100, 200, 10, 0.5) 
disp_video(VIDEO_OUT_PATH)

정성적인 측면에서 NasNet 모델이 세 가지 중에서 가장 좋은 결과를 냅니다. 그러나 정확도 면에서 2위 모델인 `faster_rcnn_resnet_101`에 비해 추론 시간이 3 배 정도 증가합니다. (다음 챕터에서 정확도를 정량화할 것입니다.) 반면 SSD는 다른 두 가지에 비해 가장 낮은 품질의 결과를 냈습니다. SSD가 유용하지 않다는 뜻은 아닙니다. SSD는 매우 효율적인 검출기지만 더욱 모델의 성능을 올리기 위해서는 대규모 데이터 세트가 필요합니다.

<a name="5-2"></a>
### 5.2 정량적 분석 - Intersection over Union

모델이 어떤 성능을 가지는지를 정량적으로 판단하기 위해서는, 적어도 검출의 관점에서는 IoU(Intersection over Union) 계산과 False Negative rate를 고려해야 합니다. 객체 검출의 경우, 검출 신뢰도가 고정 임계값 이상일 때 IoU를 계산하는 것이 현명합니다. 그렇지 않으면, 300개의 출력에서 나온 모든 경계 상자를 고려해야만 할 것입니다. 임계값을 0.5로 설정하는 것이 일반적입니다.

또한 각 모델의 프레임률(fps - 초당 프레임 수)도 고려해야 합니다. 이러한 개념들은 이전 DLI 코스에서 이미 살펴본 것들입니다.

<img src="imgs/IoU.jpg" alt="meta_arch" style="width: 600px;"/>
<p style="text-align: center;color:gray"> 그림 9. IoU 측정   </p>

여기에서는 각 프레임에서 각 정답 경계 상자를 검출할 때마다 IoU를 계산합니다. 이것은 검출기의 성능을 측정하는 좋은 방법이지만 실전에 적용할 때 필요한 중요한 정보가 하나 빠져 있습니다. 나중에 더 살펴볼 것입니다.


첫째, 우리는 각 비디오의 모든 프레임에 대한 모델, 검출, 정답 경계 상자 및 점수의 딕셔너리를 생성하는 함수를 만들어야 합니다. 아래 코드에서 커멘트 처리된 부분을 보세요. 커멘트 처리된 이유는 실행하는데 시간이 오래 걸리기 때문입니다. 따라서 본 랩에서는 모델 비교를 위해 pickle로 압축된 딕셔너리를 제공합니다.


In [0]:
def detect_frames_for_comparison(path_to_graph, path_to_labels,
                                 data_folder, min_index, max_index):
    # We load the label maps and access category names and their associated indicies
    label_map = label_map_util.load_labelmap(path_to_labels)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=1, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    
    

    # Import a graph by reading it as a string, parsing this string then importing it using the tf.import_graph_def command
    print('Importing graph...')
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(path_to_graph, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

    # Generate a video object

    print('Starting session...')
    output = []
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            # Define input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            frames_path = data_folder
            xml_path = join(data_folder, 'xml')
            num_frames = max_index - min_index
            reference_image = os.listdir(data_folder)[0]
            image = cv2.imread(join(data_folder, reference_image))
            height, width, channels = image.shape
            print('Running Inference:')
            for fdx, file_name in \
                    enumerate(sorted(os.listdir(data_folder),  key=lambda fname: int(fname.split('.')[0]) )):
                if fdx<=min_index or fdx>=max_index:
                    continue;
                image = cv2.imread(join(frames_path, file_name))
                image_np = np.array(image)
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
                bboxes = get_info_from_DF(int(file_name.split(".")[0]))
                # Actual detection.
                tic = time.time()
                (boxes, scores, classes, num) = sess.run(
                    [detection_boxes, detection_scores, detection_classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})
                toc = time.time()
                t_diff = toc - tic
                fps = 1/t_diff
                
                boxes = np.squeeze(boxes)
                classes = np.squeeze(classes)
                scores = np.squeeze(scores)
                
                vis_util.visualize_boxes_and_labels_on_image_array(
                    image,
                    boxes,
                    classes.astype(np.int32),
                    scores,
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=2,
                    min_score_thresh=0.5)


                #cv2.imwrite(join('/dli/dli-v3/iv05/data/temp', file_name),image)
                prog = '\rCompleted %.2f %%' % (100 * float(fdx - min_index + 1) / num_frames)
                print('{}'.format(prog), end = "")
                boxes = np.array([(i[0]*height, i[1]*width, i[2]*height, i[3]*width) for i in boxes])
                output.append((bboxes, (boxes, scores, classes, num, fps)))

    return output

In [0]:
PATH_TO_DATA = image_folder

model_name = 'faster_rcnn_resnet_101'
detections = detect_frames_for_comparison(models[model_name], PATH_TO_LABELS, PATH_TO_DATA, 100, 200)

두 개의 좌표값이 주어지면 `bbox_IoU` 함수가 IoU 값을 생성합니다.


In [0]:
# function to compute the intersection over union of these two bounding boxes
def bbox_IoU(A, B):
  # A = list(ymin,xmin,ymax,xmax)
  # B = list(ymin,xmin,ymax,xmax) - (xmin, ymin, xmax, ymax)
  # assign for readability 
  yminA, xminA, ymaxA, xmaxA = A
  xminB, yminB, xmaxB, ymaxB = B

  # figure out the intersecting rectangle coordinates
  xminI = max(xminA, xminB)
  yminI = max(yminA, yminB)
  xmaxI = min(xmaxA, xmaxB)
  ymaxI = min(ymaxA, ymaxB)

  # compute the width and height of the interesecting rectangle
  wI = xmaxI - xminI
  hI = ymaxI - yminI

  # compute the area of intersection rectangle (enforce area>=0)
  areaI = max(0, wI) * max(0, hI)


  # compute areas of the input bounding boxes 
  areaA = (xmaxA - xminA) * (ymaxA - yminA)
  areaB = (xmaxB - xminB) * (ymaxB - yminB)

  # if intersecting area is zero, we're done (avoids IoU=0/0 also)
  if areaI == 0: return 0, areaI, areaA, areaB

  # finally, compute and return the intersection over union
  return areaI / (areaA + areaB - areaI), areaI, areaA, areaB

이제 생성된 데이터에 대해 루프를 돌면서 `bbox_IoU` 함수를 호출하여 각 프레임에 대한 IoU 측정값을 출력합니다.


In [0]:
vid_calcs = list()
for frame_idx in range(len(detections)):
      det_boxes = detections[frame_idx][1][0]
      scores = detections[frame_idx][1][1]
      fps = detections[frame_idx][1][4]
      bbox_frame = detections[frame_idx][0]

      max_IoU_per_detection = list()

      #We loop over each bounding box and find the maximum IoU for each detection.
      for b_idx, bbox in enumerate(bbox_frame):
        IoU = 0
        for det_idx, det_box in enumerate(det_boxes):  
          if scores[det_idx] < 0.5: continue #We only include bounding box proposals with scores above and equal to 0.5         
          iou, I, A, B = bbox_IoU(det_box, bbox)
          IoU = max(iou, IoU)
        max_IoU_per_detection.append((IoU, fps))
      vid_calcs.append(max_IoU_per_detection)

검출에 대한 IoU 결과를 시각화해봅시다.


In [0]:
IoU_list=[]
for item in vid_calcs:
    IoU_list.append(item[0][0])


y_pos = np.arange(len(IoU_list))
pl.rcParams['figure.figsize'] = (18, 3)
 
plt.bar(y_pos, IoU_list, align='center', alpha=0.5)

plt.title('IoU measure for detections')
 
plt.show()

SSD 모델은 다른 두 모델에 비해 매우 낮은 IoU 값을 생성하며 검출 실패도 많이 합니다. 반면에, NasNet 모델은 훨씬 더 높은 IoU 값을 가지며 시험 데이터 세트에서 프레임을 누락하는 수도 적습니다. NasNet의 중요한 사항은 오랜 추론 시간이며, 이는 기존의 하드웨어 제한을 고려할 때 많은 온라인 애플리케이션에 적합하지 않은 모델입니다.

반면 `faster_rcnn_resnet_101`은 데이터 집합에 적용할 때 균형 잡힌 정확도와 성능을 보여줍니다. 다음 랩에서는 `전이학습 (Transfer Learning)` 기법을 이용하여 모델의 정확도를 더욱 향상시켜 보겠습니다.

<a name="6"></a>
## 6. 어노테이션을 자르고 정규화하기

Example 데이터 구조 형식으로 인코딩하기 위해서는 영상을 적당한 크기로 잘라야 합니다. 우리가 사용하고 있는 모델은 448 x 448 픽셀의 고정 너비 입력을 필요로 합니다. 즉, 객체 검출 API를 통해 이 모델에 공급되는 모든 이미지는 448 x 448 픽셀 크기로 변경된다는 뜻입니다. 원본 영상의 가로 세로 비율이나 높은 해상도 때문에 모델의 전처리 단계에서 데이터를 많이 변경해야 하는데, 이 때문에 결과가 좋지 않게 나올 가능성도 있습니다. 목표 크기가 전체 이미지에 비해 상대적으로 작기 때문에 크기 조절 과정에 민감하게 반응할 수도 있습니다. 전체 크기를 줄이는(Resizing) 대신에 영상의 원래 해상도를 유지하면서 영상을 잘라내면(Cropping) 영상 품질과 관심 목표 관점에서 의도하지 않은 크기 조절 부작용을 제거할 수 있습니다. 그러나 이 방법은 데이터 처리를 약간 더 복잡하게 만듭니다. 아래의 함수는 이미지 내 잘라낼 값을 결정하고, 잘라내는 영역이 아닌 이외의 영역의 어노테이션을 제외하는 데 필요한 변경사항을 계산하며, 남은 어노테이션을 새로 잘라낸 이미지 좌표계로 다시 인덱싱합니다. 경계 상자의 값들은 이미지 높이와 너비에 비례하여 정규화되고 데이터를 더욱 유연하게 만들어 향후에 재사용이 편리하도록 합니다. 여기에서는 영상의 가운데를 중심으로 자르기를 했지만 어디를 자를지에 대한 제약사항은 없습니다.

이러한 선택은 이 모델에 대해 구축된 추론 파이프라인에서 다운스트림 (Downstream) 영향을 미친다는 점에 유의해야 합니다. 훈련 데이터와 마찬가지로, 추론을 위해 그래프를 통과하는 모든 데이터도 비슷한 크기를 가져야 합니다. 분명히, 센서는 우리 모델에 의해 제한된 범위보다 훨씬 더 큰 시야에서 정보를 수집하며, 이와 비슷한 모델을 제공합니다. 이러한 객체 검출기를 목표로 하는 추론 파이프라인을 만들 경우, 파이프라인은 분할, 중첩 및 관련된 모든 어노테이션 처리를 해야할 필요가 있습니다.

편의성을 위해 DataFrame `width` and `height` 컬럼을 추가할 것입니다.


In [0]:
img_height = 692
img_width = 882
annotated_frames.insert(1, 'width', img_width)
annotated_frames.insert(1, 'height', img_height)

아래에서는 각 이미지를 자르는 크기를 정의합니다. 이미지 자르기 크기를 설정하는 함수를 구현할 수 있는 많은 방법이 있습니다. 예를 들어 모든 이미지를 가운데 주위로 자르고 DataFrame을 필터링하여 이동 중인 차량이 없는 프레임을 제거할 수 있습니다. 한 걸음 더 나아가서 각 프레임 내에서 움직이는 객체의 가운데를 중심으로 자르기 영역을 조정합니다. 좌표가 영상 경계 내에 들어오는지 확인하고 잘못된 경계 상자 (음수값을 가진 좌표)가 생성되지 않도록 합니다.


In [0]:
# define the crop size which is equal to the input size of our neural network
g_image_size = (448.0, 448.0)

def set_crop_size(crop_size, frames):
    for i, box in frames.iterrows():
        center_box_x = int (box['xmin'] + (box['xmax'] - box['xmin']) / 2)
        center_box_y = int (box['ymin'] + (box['ymax'] - box['ymin']) / 2)

        start_x = center_box_x - crop_size[0] / 2    
        end_x = start_x + crop_size[0]
        if start_x < 0:
            if box['xmin'] - 5 >= 0:
                start_x = box['xmin'] - 5
            else:
                start_x = box['xmin']
            end_x = start_x + crop_size[0]
        elif end_x >= box['width']:
            end_x = box['width']
            start_x = end_x - crop_size[0]

        start_y = center_box_y - crop_size[1] / 2    
        end_y = start_y + crop_size[1]
        if start_y < 0:
            if box['ymin'] - 5 >= 0:
                start_y = box['ymin'] - 5
            else:
                start_y = box['ymin']
            end_y = start_y + crop_size[1]
        elif end_y >= box['height']:
            end_y = box['height']
            start_y = end_y - crop_size[1]

        frames.at[i,'crop'] = [(start_x, start_y, end_x, end_y)]
    return frames


정규화된 데이터에서 또 다른 문제는 객체의 크기가 자르기 크기보다 큰 경우입니다. 이런 경우 객체를 자를 수 없기 때문에 적절한 망 입력을 만들기 위한 의사결정을 해야만 합니다.

우리의 샘플 비디오에서 그러한 사례는 차량이 프레임 하단에 도달하면 발생합니다. 아래 예를 참조하십시오.


<img src="imgs/resize_samples.jpg" alt="Vatic imaging"/>
<p style="text-align: center;color:gray"> 그림 10. 망 입력 크기보다 큰 경계 상자를 가지는 차량</p>


이 문제를 해결하기 위한 우리의 선택은 차량의 크기가 입력 크기보다 큰 프레임의 크기를 줄인 후에 자르기 작업을 수행하는 것입니다. 또 프레임의 크기가 편의상 조절됐는지를 나타내는 `resize` 칼럼을 새로 추가합니다. 디폴트값은 False입니다.

In [0]:
annotated_frames.insert(1, 'resize', False)

DataFrame을 살펴봅시다.

In [0]:
annotated_frames.head()

 
<a name="e4"></a>
### 연습 4

우리의 데이터에서 불필요한 컬럼을 삭제할 것입니다. 아래의 코드는 `occluded` 컬럼을 삭제하도록 설정되어 있습니다. `outside` 컬럼을 제거하는 코드를 추가하십시오.

In [0]:
# Remove the unnecessary columns since they are all the same value now
annotated_frames = annotated_frames.drop("occluded", axis=1)
annotated_frames.head()
# <<<<<<<<<<<<<<<<<<<<YOUR CODE HERE >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


경계 상자의 높이와 너비을 바탕으로 정상 크기의 경계 상자를 포함하는 `normal_size_frames`와 지나치게 큰 경계 상자를 포함하는 `oversized_frames` 등, 두 세트의 DataFrames을 만들 것입니다.


In [0]:
normal_size_frames = annotated_frames[annotated_frames.apply(lambda x: x['xmax'] - x['xmin'] <= g_image_size[0], axis=1) &
                                     annotated_frames.apply(lambda x: x['ymax'] - x['ymin'] <= g_image_size[1], axis=1)]

oversized_frames = annotated_frames[annotated_frames.apply(lambda x: x['xmax'] - x['xmin'] > g_image_size[0], axis=1) |
                                     annotated_frames.apply(lambda x: x['ymax'] - x['ymin'] > g_image_size[1], axis=1)]                                 

print("Number of frames within the crop size:{}, number of oversized vehicles/frames: {}".format(len(normal_size_frames),
                                                                                                 len(oversized_frames)))

normal_size_frames = set_crop_size(g_image_size, normal_size_frames)
normal_size_frames.head()

프레임의 크기 조절은 경계 상자 어노테이션 값(xmin, ymin, xmax 및 ymax)에 영향을 미칩니다. 이러한 값은 크기 조절 비율에 따라 변경할 필요가 있습니다. 프레임 내에 객체를 포함하려면 `max(bounding_box_width, bounding_box_height) + some_offset_value` 를 계산하여 크기 조절 비율로 삼습니다. 마지막으로, 그에 따라 __bounding_box__ 좌표를 변경합니다.


In [0]:
for i, box in oversized_frames.iterrows():
    resize_ratio = 0.0
    diff_x = box['xmax'] - box['xmin'] + 50 # adding offset to prevent round up errors
    diff_y = box['ymax'] - box['ymin'] + 50
    
    #find the maximum of x and y required ratio reduction
    resize_ratio = g_image_size[0]/max(diff_x, diff_y)
    
    #correct the existing bounding box values according to the ratio
    oversized_frames.at[i,'xmin'] = int(box['xmin'] * resize_ratio) 
    oversized_frames.at[i,'xmax'] = int(box['xmax'] * resize_ratio) 
    oversized_frames.at[i,'ymin'] = int(box['ymin'] * resize_ratio) 
    oversized_frames.at[i,'ymax'] = int(box['ymax'] * resize_ratio)
    
    #correct height and width values and set the resize column value to True
    oversized_frames.at[i,'width'] = int(box['width'] * resize_ratio)
    oversized_frames.at[i,'height'] = int(box['height'] * resize_ratio)
    oversized_frames.at[i,'resize'] = True

oversized_frames = set_crop_size(g_image_size, oversized_frames)    
oversized_frames.head()

얼마나 많은 오버사이즈 프레임이 생성되었는지 아래에서 볼 수 있습니다.


In [0]:
print('Number of oversized frames:',  len(oversized_frames))

획득한 값이 여전히 유효하고, 반올림 에러가 결과에 영향을 미치지 않았는지 확인해야 합니다. 또 영상 좌표가 잘린 영역으로 바뀌기 때문에 경계 상자 좌표에서 잘린 박스의 상단 좌표와 왼쪽 좌표를 빼줘야 합니다.


In [0]:
def normalize_frames(frames):
    normalized_frames = frames[frames.apply(lambda x: x['crop'][0][0] >= 0, axis=1) &
                                     frames.apply(lambda x: x['crop'][0][1] >= 0, axis=1) &
                                     frames.apply(lambda x: x['crop'][0][2] <= x['width'], axis=1) &
                                     frames.apply(lambda x: x['crop'][0][3] <= x['height'], axis=1) &
                                     frames.apply(lambda x: x['crop'][0][0] <= x['xmin'], axis=1) &
                                     frames.apply(lambda x: x['crop'][0][1] <= x['ymin'], axis=1) &
                                     frames.apply(lambda x: x['crop'][0][2] >= x['xmax'], axis=1) &
                                     frames.apply(lambda x: x['crop'][0][3] >= x['ymax'], axis=1)]
    for i, box in normalized_frames.iterrows():
        normalized_frames.at[i, 'xmin'] = box['xmin'] - int(box["crop"][0][0])
        normalized_frames.at[i, 'ymin'] = box['ymin'] - int(box["crop"][0][1])
        normalized_frames.at[i, 'xmax'] = box['xmax'] - int(box["crop"][0][0])
        normalized_frames.at[i, 'ymax'] = box['ymax'] - int(box["crop"][0][1])
        
    return normalized_frames

In [0]:
cropped_frames = normalize_frames(normal_size_frames)
print('Number of normal sized objects:',  len(cropped_frames))

In [0]:
cropped_frames_oversize = normalize_frames(oversized_frames)
print('Number of oversized objects:',  len(cropped_frames_oversize))

`normal_size_frames` 와 `oversized_frames`에서 샘플을 추출해서 도표를 그리고 비교합니다. 8 개의 자른 영상 샘플이 아래에 나와 있습니다.


<img src="imgs/sample8.jpg" alt="Vatic imaging"/>
<p style="text-align: center;color:gray"> 그림 11. 무작위로 선택한 프레임 예</p>


이러한 이미지를 읽고, 자르고, 도표로 그리려면 몇 가지 보조 함수가 필요합니다.


- __crop_image__:          입력 참조 좌표를 기반으로 크기가 조절된 이미지를 리턴합니다.
- __showarray__:           잘린 이미지를 나타내는 배열을 그립니다.
- __draw_rectangle__:      각 샘플에서 매칭된 차량의 경계 상자를 그립니다.
- __plot_random_samples__: DataFrame의 샘플을 얻고 각각의 항목에 위 함수들을 적용합니다.


In [0]:
from IPython.display import clear_output, Image, display
from io import StringIO
import PIL.Image
# Helper function to crop images
def crop_image(pil_image, coordinates):
    # get the size of the image
    xmin, ymin, xmax, ymax = int(coordinates[0]), int(coordinates[1]), int(coordinates[2]), int(coordinates[3])    
    crop_img = pil_image[ymin:ymax, xmin:xmax]
    return crop_img

def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    
def draw_rectangle(draw, coordinates, color, width=1):
    for i in range(width):
        rect_start = (coordinates[0][0] - i, coordinates[0][1] - i)
        rect_end = (coordinates[1][0] + i, coordinates[1][1] + i)
        draw.rectangle((rect_start, rect_end), outline = color)


`plot_random_samples` 은 주어진 집합에서 8개의 무작위 샘플을 선택하고 각 표본에 대해 주어진 DataFrame에서 정답 데이터를 찾고, 프레임 상에 박스를 그린 후, 최종적으로 결과 샘플을 그립니다.



In [0]:
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
from matplotlib.pyplot import imshow

def plot_random_samples(frames):
    sample_frames = frames.sample(n=8)
    fig=plt.figure(figsize=(15, 8))
    columns = 4
    rows = 2
    i = 1 

    for index, box in sample_frames.iterrows():
        #print(box["crop"])

        im = Image.open('{}/images/{}/{}.jpg'.format(config["Base_Dest_Folder"], config["Test_Video_ID"], box["frame_no"]))


        if box['resize']:
            im = im.resize((int(box['width']), int(box['height'])), Image.ANTIALIAS)

        xmin, ymin, xmax, ymax = box["xmin"], box["ymin"], box["xmax"], box["ymax"]


        cropped_im = im.crop(box["crop"][0])


        draw = ImageDraw.Draw(cropped_im)
        draw.rectangle(((xmin, ymin), (xmax, ymax)), fill=None, outline='red')
        draw_rectangle(draw, ((xmin, ymin), (xmax, ymax)), color=colors[2], width=3)


        fig.add_subplot(rows, columns, i)
        i += 1
        plt.imshow(np.asarray(cropped_im))
    plt.show()


첫째, 크기가 너무 큰 DataFrame의 샘플을 그립니다.


In [0]:
plot_random_samples(cropped_frames_oversize)

다음으로, 정상 프레임의 샘플을 그립니다.


In [0]:
plot_random_samples(cropped_frames)

두 프레임 집합과 최종 데이터 세트를 합치고 샘플을 다시 그립니다.


In [0]:
temp_frames = [cropped_frames_oversize, cropped_frames]
cropped_frames = pd.concat(temp_frames)

조합된 집합에서 샘플 영상을 그려 봅시다.


In [0]:
plot_random_samples(cropped_frames)

이 단계에서, 우리는 이미지를 자르고 주변 경계 상자를 그리는 데 필요한 모든 프레임 정보를 포함하는 DataFrame을 가지고 있습니다. 노이즈 데이터는 필터링되고, 지나치게 큰 프레임은 축소되어, 포함된 차량이 우리 모델의 입력 스키마에 맞도록 조절되었습니다.

데이터 준비는 성공적인 IVA 애플리케이션 구축을 위한 가장 길고 가장 중요한 작업이 될 수 있으며, 대개 데이터 유형, 카메라, 조명, 날씨 등에 매우 영향을 받습니다. 이제는 __TFRecords__ 를 만드는 다음 단계로 넘어갈 준비가 되었습니다.


<a name="7"></a>
## 7. TFRecord 파일 생성하기

TFRecord는 TensorFlow 데이터 집합을 저장하는 이진 형식입니다. 이를 통해 데이터를 압축해서 표현할 수 있을 뿐 아니라 데이터 검색 및 메모리 관리 성능도 향상시킬 수 있습니다. 어노테이션 좌표를 정규화했으니 이제 데이터를 TFRecord 안에 넣어봅시다. 함수를 각각 살펴본 후, 마지막에 그것을 모두 합쳐서 처리할 것입니다.

TFRecords는 데이터를 이진 형식으로 저장하기 때문에 `구조화된` 형식으로 데이터를 제공해야 합니다. TensorFlow는 데이터 구조를 TFrecords로 직렬화하는 두 가지 함수를 제공합니다. `tf.trian.Example` 와 `tf.train.SequenceExample`이  그 두 개인데, {"string": tf.train.Feature} 매핑을 제공하여 데이터를 TensorFlow 표준 모델로 변환합니다.


TFRecords 를 만들 때 중요한 고려사항은 경계 상자 좌표를 0과 1 사이의 float 값으로 표준화해야 한다는 점입니다. 또한 앞에서 설명한 바와 같이, Example 데이터 구조로 인코딩되기 전에 이미지를 잘라내야 합니다. 다음 함수는 TFRecords를 생성하고 `crop` 좌표를 이용하여 이미지를 자릅니다.



In [0]:
from PIL import *
# test_output = {}
def To_tf_example(frame_data, img_path, img_name, 
                         label_map_dict,
                         img_size,
                         single_class):
    

    pil_image = Image.open(os.path.join(img_path,img_name))
    
    
    if frame_data['resize']:
        pil_image = pil_image.resize((int(frame_data['width']), int(frame_data['height'])), Image.ANTIALIAS)
    
    cropped_im = pil_image.crop(frame_data["crop"][0])

    encoded = cv2.imencode('.jpg', np.asarray(cropped_im))[1].tostring()

    xmin = []
    ymin = []
    xmax = []
    ymax = []
    classes = []
    classes_text = []

    
    
    # Append the  coordinates to the overall lists of coordinates
    xmin.append(float(frame_data['xmin'])/float(img_size[0]))
    ymin.append(float(frame_data['ymin'])/float(img_size[1]))
    xmax.append(float(frame_data['xmax'])/float(img_size[0]))
    ymax.append(float(frame_data['ymax'])/float(img_size[1]))
    


    # If only detecting object/not object then ignore the class-specific labels
    if single_class:
        classes.append(1)
    else:
        class_name = frame_data['label']
        classes_text.append(class_name.encode('utf8'))
        classes.append(label_map_dict[class_name])

    # Generate a TF Example using the object information
    example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(int(img_size[1])),
        'image/width': dataset_util.int64_feature(int(img_size[0])),
        'image/filename': dataset_util.bytes_feature(
            img_name.encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(
            img_name.encode('utf8')),
        'image/filepath': dataset_util.bytes_feature(
            img_path.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded),
        'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes)}))
    return example

<a name="7-1"></a>
### 7.1 어노테이션과 영상을 TensorFlow Example들로 인코딩하기

`generate_tf_records` 함수는 이미지에 특화된 DataFrames을 입력으로 받습니다. 가려지거나, 사라졌거나, 잘린 영역 바깥에 있는 객체들을 필터링하는 것에 대해 이전 섹션에서 배웠는데, 이들 DataFrames은 그러한 필터링 과정으로부터 얻어집니다. 아래 함수는 `To_tf_example` 함수를 이용하여 원본 영상을 자릅니다. 이 데이터는 jpeg으로 인코딩되어 다음 단계에서 TFRecord에 기록됩니다. 프레임과 관련된 각각의 어노테이션, 그리고 컬링 (Culling)에서 살아남은 어노테이션도 함께 기록됩니다.

In [0]:
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util
def generate_tf_records(writer, 
                        frames_df,
                        image_folder,
                        reference_frames,
                        label_map_dict):

    for index, the_item in frames_df.iterrows():
        #check if frame belongs to the reference set; i.e. test/train
        if int(the_item["frame_no"]) in reference_frames:
            
            print("\r frame: {:>6}".format(int(the_item["frame_no"])), end='\r', flush=True)
            file_name = "{}.jpg".format(the_item["frame_no"])
            tf_example = To_tf_example(the_item,image_folder, file_name, label_map_dict, g_image_size, False)
            
            writer.write(tf_example.SerializeToString())

<a name="7-2"></a>
### 7.2 훈련 및 검증 데이터 만들기 

훈련 과정을 지원하기 위해, 우리는 훈련 셋을 포함한 레코드와 검증 셋을 포함한 또 다른 레코드를 만들어야 합니다. 훈련 집합의 셋은 학습 단계를 진행하면서 데이터에 가장 적합한 모델 매개변수(가중치 및 편향)를 결정하기 위해 사용됩니다. 검증 집합의 셋은 모델의 평가를 수행하고 훈련 중인 모델의 현재 성능을 평가하는 데에 사용됩니다. 최종적으로 훈련된 모델을 시험하기 위해서는 테스트 데이터 (학습데이터로 사용되지 않은) 집합을 준비해 두어야 합니다.

검증 데이터는 모델의 현재 성능을 나타내는 지표로 작용하며, 모델이 과적합됐거나 너무 빨리 수렴되었는지 여부를 나타내는 데에 사용됩니다. 관찰 중인 결과 모델에 대해 긍정적 평가가 나오든 부정적 평가가 나오든, 연구자가 훈련 과정을 조기에 종료하는 데에 도움을 줍니다. 조기 종료를 하게 되면 긍정적인 경우 최종 모델이 되는 것이고 부정적인 경우 설정을 변경하여 학습을 다시 시작합니다. 따라서 검증 데이터는 실제로 Low-level 학습에 영향을 미치거나 최종 평가에 기여하지 않지만 학습 과정에 중요한 영향을 미치는 데이터 집합입니다.
  
우리가 방금 만든 데이터 프레임은 단일 영상의 모든 어노테이션을 가진 데이터를 포함하고 있습니다. `Pandas` 라이브러리의 장점 중 하나는 다른 데이터 조작 라이브러리의 입력으로 사용할 수 있다는 점입니다. 우리는 `scikit-learn`를 이용하여 데이터를 훈련셋과 검증 셋으로 분할 비율에 따라 나눌 것입니다.



In [0]:
import ast
from sklearn.model_selection import train_test_split
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import dataset_util
from random import shuffle

unique_frames = cropped_frames.frame_no.unique()
#shuffle and split the set        
shuffle(unique_frames)
split = 0.2
train, test = train_test_split(unique_frames, test_size=split)

<a name="7-3"></a>
### 7.3 함수들을 연결하여 TFRecord 만들기

이제 모든 것을 합쳐서 TFRecord 파일을 생성하겠습니다. 다음과 같은 단계를 거치게 됩니다.

1. 레이블 맵 파일(아래 설명)을 딕셔너리 형식으로 읽어들입니다. 이 작업은 API에서 제공하는 유틸리티를 사용하여 손쉽게 할 수 있습니다.
2. 모든 데이터의 상위 디렉토리를 가져옵니다.
3. 생성된 레코드 파일의 출력 경로를 만듭니다.
4. 각 DataFrame 그룹(훈련 및 테스트용)을 `generate_tf_records` 함수로 전달합니다.
5. 어노테이션을 프레임별로 기록하여 TensorFlow Example을 만듭니다.
6. 생성된 Example을 TFRecord 파일에 저장합니다.

이 모든 함수을 한데 묶어 훈련 및 검증 데이터용 TFRecord를 생성하려면 특정 형식의 레이블 맵이 필요합니다. 이 파일은 기본적으로 클래스 ID를 클래스 이름에 매핑합니다. 여러분은 제공된 파일을 사용하면 되는데 아래 코드에서 그 파일을 이용하고 있습니다.

파일의 내용은 아래처럼 보일 것입니다.

```json
item {
  id: 1
  name: 'Object'
}
```

랩 환경에서의 시간과 컴퓨팅 제약으로 인해 단일 영상의 데이터로만 작업하고 있다는 것을 기억하십시오. 이러한 접근 방식은 더 큰 데이터 셋으로 확장할 수 있습니다. 이후의 랩에서는 모델을 훈련시키기 위해 더 큰 레코드 파일을 같은 방식으로 만들게 될 것입니다.

아래 과정은 시간이 좀 걸립니다. 다음 랩의 내용을 미리 읽기에 좋은 기회입니다.



In [0]:
video_list = ['126_206-A0-3']
label_map_dict = label_map_util.get_label_map_dict(config["Label_Map"])
train_writer = tf.python_io.TFRecordWriter(join(config["Base_Dest_Folder"],'train.record'))
eval_writer = tf.python_io.TFRecordWriter(join(config["Base_Dest_Folder"],'eval.record'))
for xx in video_list:
    #create train record
    generate_tf_records(train_writer, 
                        cropped_frames,
                        '{}/images/{}'.format(config["Base_Dest_Folder"], config["Test_Video_ID"]),
                        train,
                        label_map_dict)
    #create eval record
    generate_tf_records(eval_writer, 
                        cropped_frames,
                        '{}/images/{}'.format(config["Base_Dest_Folder"], config["Test_Video_ID"]),
                        test,
                        label_map_dict)
        
train_writer.close()   
eval_writer.close()   

## 정리

IVA 과정의 첫 번째를 수료한 것을 축하합니다! 시간이 있다면 위의 스크립트를 수정하여 다른 모델과 비디오를 조합하여 실험해보세요.

지금까지 우리는 다음과 같은 것들을 배웠습니다.
* 몇 가지 객체 검출 방법와 그들의 차이점과 장단점
* 비디오 프레임과 어노테이션을 TensorFlow 객체 검출 API와 메트릭 정의에 쉽게 사용할 수 있는 형식으로 변환하기
* 모델 정확도를 정성적으로 평가하기 위한 출력을 이해하는 방법
* IoU 지표를 이용하여 객체 검출 모델의 정확도 및 성능을 정량적으로 측정하는 방법

다음 랩에서는 모델을 훈련시키고 네트워크 가중치를 파인튜닝 (Fine-tuning)하는 방법을 배울 것입니다. 또한 비디오에서 객체를 추적하는 방법을 배울 것입니다. 참여해 주셔서 감사합니다!



## 해답
<a name="a1"></a>
### 연습 1:

In [0]:
print("Ratio of frames with moving vehicles to tatal: {0:.2f}%".format((frame_existance == 1.0).sum() / len(frame_existance) * 100))

앞으로 돌아가시려면 [여기](#e1)를 클릭하세요.

<a name="a2"></a>
### 연습 2:

In [0]:
sedans = annotated_frames[annotated_frames["attributes"].str.contains("sedan") == True]
print ('Total number of sedans: {}'.format(len(sedans)))

앞으로 돌아가시려면 [여기](#e2)를 클릭하세요.

<a name="a3"></a>
### 연습 3:

In [0]:
# YOUR CODE GOES HERE
def calc_average_HW(row):
    row['Average_Height'] = row['ymax'] - row['ymin']
    row['Average_Width'] = row['xmax'] - row['xmin']
    return row
Average_HW = inside_items.groupby(['track_id']).apply(calc_average_HW)
Average_HW = Average_HW.groupby(['track_id']).mean()

Average_HW.head()

앞으로 돌아가시려면 [여기](#e3)를 클릭하세요.